In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install accelerate==0.25.0
!pip install bertopic==0.15.0
!pip install datasets==2.14.4
!pip install faiss-cpu==1.7.4
!pip install langchain==0.0.348
!pip install langchainhub==0.1.14
!pip install sentence-transformers==2.2.2
!pip install sentencepiece==0.1.99
!pip install transformers==4.24.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.3
    Uninstalling accelerate-0.29.3:
      Successfully uninstalled accelerate-0.29.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 39.0 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl.metadata (3.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 9.1 MB/s eta 0:00:00
Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=819466 sha256=147a8bdf

In [2]:
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, PreTrainedTokenizer, PreTrainedTokenizerFast
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
import numpy as np
import typing as tp
imdb = load_dataset("imdb", split="test")



Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
imdb_df = imdb.to_pandas()
imdb_sample = imdb_df.sample(n=1000, random_state=2023)

In [6]:
model_name = "siebert/sentiment-roberta-large-english"

model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
config = transformers.AutoConfig.from_pretrained(model_name)

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [7]:
class ExampleDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [8]:
example_dataset = ExampleDataset(
    texts=imdb_sample["text"].tolist(),
    labels=imdb_sample["label"].tolist(),
    tokenizer=tokenizer,
    max_length=512
    )

In [ ]:
import typing as tp

import pandas as pd
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    """
    This class designs logic to retrieve data from a custom dataset.
    According to pytorch Dataset conception any map style dataset
    should implement at least __len__ and __getitem__ methods.
    """

    def __init__(
        self,
    ) -> None:
        """
        YOUR CODE HERE
        """
        pass

    def __len__(self) -> int:
        """
        returns number of rows in data
        """

        """
        YOUR CODE HERE
        """
        pass

    def __getitem__(self, idx: int) -> tp.Dict[str, tp.Any]:
        """
        retrieves data for single index.
        may include data processing and transformations.
        E.g. augmenting data or tokenizing it.
        returns dict with keys "input_ids", "label" and probably some more metadata (you decide whethere you need something more here)
        """

        """
        YOUR CODE HERE
        """
        pass


class ModelTrainer:
    """
    This class implements logic run an experiemnt with a provided transformers classification model.
    It incudes following components:
    - load data
    - load and configure a model and its artifacts
    - train model
    - validate model
    - save model
    - compue metrics
    - run_experiment (as the man entrypoint to execute all flow)

    Attention: current module intentionally doesnt support model inference or model serving.
    It is a good practice to separate train/inference classes otherwise it is hard to maintain it all.

    """

    def __init__(self):
        """
        YOUR CODE HERE
        """
        pass

    def configure_optimizer(self) -> None:
        """
        adds a self.optimizer attribute with a chosen optimizer and its params.
        """

        """
        YOUR CODE HERE
        """
        pass

    def configure_scheduler(self) -> None:
        """
        adds a self.scheduler attribute with a chosen scheduler (e.g. ReduceLROnPlateau).
        """

        """
        YOUR CODE HERE
        """
        pass

    def apply_data_parallel(self) -> None:
        """
        checks number of available cuda devices,
        if number of GPUs is > 1, moves self.model to a DataParallel state for faster training.
        """

        """
        YOUR CODE HERE
        """
        pass

    def load_data(self, filename: str, split: str) -> pd.DataFrame:
        """
        uses Datasets library to load a dataset, takes as input dataset name (e.g. "imdb")
        and a split. Loads data into pandas.
        """

        """
        YOUR CODE HERE
        """
        pass

    def train(self, dataset: CustomDataset) -> None:
        """
        YOUR CODE HERE
        """
        pass

    def validate(self, dataset: CustomDataset) -> tp.Dict[str, tp.Iterable]:
        """
        takes a trained model and runs it on validation data.
        Returns a dict with the keys "valid_labels" and "valid_preds" and corresponding values.
        """

        """
        YOUR CODE HERE
        """
        pass

    def compute_metrics_report(
        self, labels: tp.Iterable, predictions: tp.Iterable
    ) -> tp.Any:
        """
        Computes classification metric (or several metrcis) for given task.
        """

        """
        YOUR CODE HERE
        """
        pass

    def save_model(self, dst_path: str) -> None:
        """
        Saves model to dst_path. Be careful to check if a model is on DataParallel state.
        If it is, one needs to process it accordingly.
        """

        """
        YOUR CODE HERE
        """
        pass

    def run_experiment(self):
        """
        Main entrypoint.
        Runs the flow from loading data to computing metrics.
        """

        """
        YOUR CODE HERE
        """
        pass


if __name__ == "__main__":
    """run experiment"""
    model_trainer = ModelTrainer(...)
    model_trainer.run_experiment()